# Basics

Import libraries and functions.

In [1]:
import pandas as pd
import numpy as np
import glob
import os
from pyspark.sql.functions import concat, col, lit, split

Firstly we load the database from World Data Bank that has been downloaded and extracted in the *Data extraction* notebook. We acquire it from the predetermined path that is on our computer.

In [9]:
df= pd.read_csv (os.getcwd()+'\WDIData.csv')
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.936004,17.337896,17.687093,18.140971,18.491344,18.825520,19.272212,19.628009,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.499471,6.680066,6.859110,7.016238,7.180364,7.322294,7.517191,7.651598,NaN,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.855399,38.046781,38.326255,38.468426,38.670044,38.722783,38.927016,39.042839,NaN,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.794160,32.001027,33.871910,38.880173,40.261358,43.061877,44.270860,45.803485,NaN,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,18.663502,17.633986,16.464681,24.531436,25.345111,27.449908,29.641760,30.404935,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384365,Zimbabwe,ZWE,Women who believe a husband is justified in be...,SG.VAW.REFU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,14.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384366,Zimbabwe,ZWE,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.700000,NaN,NaN,NaN,5.418352,NaN,NaN,NaN
384367,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,33.500000,32.400000,NaN,NaN,NaN,33.658057,NaN,NaN,NaN
384368,Zimbabwe,ZWE,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,59.200000,59.400000,59.500000,59.700000,59.900000,60.000000,60.200000,60.400000,NaN,NaN


Moreover, to work more comfortably, we remove those columns not useful for us, as *Country Name* and *Indicator Code*, since with the *Country Code*, *Value* and the *Indicator Name* we have the information relevant.

In [10]:
df.drop(columns=["Country Name","Indicator Code"], axis=1, inplace=True)

From the, almost, two hundred countries we have information about in the worldwide database, we have decided to study 50 of them, grouping them by geographical and economical similiarities. With this, we can keep in our dataframe the selected countries.

In [11]:
europe_list=['DEU','FRA','SWE','GBR','ESP','HRV','POL','GRC','AUT','NLD']
persian_list=['IRQ','QAT','ARE','SAU','AZE','YEM','YDR','OMN']
naf_list=['DZA','EGY','LBY','ISR','TUR','MAR']
saf_list=['SEN','ZAF','LBR','MOZ','CMR','NGA','GHA']
asia_list=['BGD','IND','VNM','THA','IDN','PHL','KOR']
latam_list=['MEX','BRA','ARG','PER','VEN','COL','CHL','PCZ','CRI']
two_list=['USA','CHN']
country_list=europe_list+persian_list+naf_list+saf_list+asia_list+latam_list+two_list 

In [12]:
df1=df.loc[df['Country Code'].isin(country_list)]

Now we transpose the rows of years into the columns.

In [13]:
df2=(df1.set_index(["Country Code", "Indicator Name"]).stack().reset_index(name='Value').rename(columns={'level_2':'Date'}))
df2

,Country Code,Indicator Name,Date,Value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2
...,...,...,...,...
1729300,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0
1729301,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0
1729302,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0
1729303,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0


Our time range covers from 1960 to 2021. However, the record is not uniform and complete for all areas and indicators. We can appreaciate that specially in the first years of the last century, so many data is missing, so it makes no sense to study it. Besides, for the year 2021 many data is also lacking. Therefore, we would delimit our study between 1990 and 2020.

In [14]:
df2[['Date']] = df2[['Date']].astype(int)

In [15]:
df2.dtypes

Country Code       object
Indicator Name     object
Date                int32
Value             float64
dtype: object

In [20]:
df3 = df2[df2['Date'] > 1989]
df3

,Country Code,Indicator Name,Date,Value
0,DZA,Access to clean fuels and technologies for coo...,2000,97.1
1,DZA,Access to clean fuels and technologies for coo...,2001,97.3
2,DZA,Access to clean fuels and technologies for coo...,2002,97.8
3,DZA,Access to clean fuels and technologies for coo...,2003,98.0
4,DZA,Access to clean fuels and technologies for coo...,2004,98.2
...,...,...,...,...
1729300,YEM,Young people (ages 15-24) newly infected with HIV,2016,200.0
1729301,YEM,Young people (ages 15-24) newly infected with HIV,2017,200.0
1729302,YEM,Young people (ages 15-24) newly infected with HIV,2018,200.0
1729303,YEM,Young people (ages 15-24) newly infected with HIV,2019,200.0


In [21]:
indicators_list=['GDP (current US$)','Literacy rate, adult total (% of people ages 15 and above)', 'Government expenditure on education, total (% of government expenditure)','Net migration','Commercial service exports (current US$)','Exports of goods and services (current US$)','Taxes on international trade (current LCU)','Fertility rate, total (births per woman)','People using at least basic sanitation services (% of population)','Employment in agriculture (% of total employment) (modeled ILO estimate)','Employment in services (% of total employment) (modeled ILO estimate)','Employment in industry (% of total employment) (modeled ILO estimate)','Electricity production from renewable sources, excluding hydroelectric (kWh)','Number of infant deaths','Number of infant deaths','Foreign direct investment, net (BoP, current US$)','Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)','Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)','Research and development expenditure (% of GDP)','Labor force with advanced education (% of total working-age population with advanced education)','Suicide mortality rate (per 100,000 population)','CPIA gender equality rating (1=low to 6=high)','Share of youth not in education, employment or training, total (% of youth population)','Government expenditure on education, total (% of government expenditure)']

In [22]:
df4=df3.loc[df3['Indicator Name'].isin(indicators_list)]
pd.set_option('display.max_rows', 10)
df4

,Country Code,Indicator Name,Date,Value
5334,DZA,Commercial service exports (current US$),1990,4.795977e+08
5335,DZA,Commercial service exports (current US$),1991,3.747657e+08
5336,DZA,Commercial service exports (current US$),2005,2.466000e+09
5337,DZA,Commercial service exports (current US$),2006,2.512000e+09
5338,DZA,Commercial service exports (current US$),2007,2.786733e+09
...,...,...,...,...
1727918,YEM,Total alcohol consumption per capita (liters o...,2000,7.900000e-01
1727919,YEM,Total alcohol consumption per capita (liters o...,2005,3.400000e-01
1727920,YEM,Total alcohol consumption per capita (liters o...,2010,1.800000e-01
1727921,YEM,Total alcohol consumption per capita (liters o...,2015,5.500000e-02


In [23]:
df5=df4.set_index(["Country Code", "Date"]).pivot(columns="Indicator Name", values="Value").reset_index()
df5

Indicator Name,Country Code,Date,CPIA gender equality rating (1=low to 6=high),Commercial service exports (current US$),"Electricity production from renewable sources, excluding hydroelectric (kWh)",Employment in agriculture (% of total employment) (modeled ILO estimate),Employment in industry (% of total employment) (modeled ILO estimate),Employment in services (% of total employment) (modeled ILO estimate),Exports of goods and services (current US$),"Fertility rate, total (births per woman)",...,"Literacy rate, adult total (% of people ages 15 and above)","Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)",Net migration,Number of infant deaths,People using at least basic sanitation services (% of population),Research and development expenditure (% of GDP),"Share of youth not in education, employment or training, total (% of youth population)","Suicide mortality rate (per 100,000 population)",Taxes on international trade (current LCU),"Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)"
0,ARE,1990,NaN,NaN,0.0,NaN,NaN,NaN,NaN,4.454,...,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ARE,1991,NaN,NaN,0.0,8.46,33.330002,58.200001,NaN,4.253,...,NaN,NaN,NaN,645.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ARE,1992,NaN,NaN,0.0,8.37,33.360001,58.279999,NaN,4.041,...,NaN,NaN,368126.0,618.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ARE,1993,NaN,NaN,0.0,8.24,33.470001,58.290001,NaN,3.827,...,NaN,NaN,NaN,592.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,1994,NaN,NaN,0.0,8.13,33.490002,58.380001,NaN,3.618,...,NaN,NaN,NaN,568.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,ZAF,2017,NaN,1.614806e+10,NaN,5.28,23.340000,71.379997,1.042884e+11,2.430,...,87.046669,NaN,727026.0,32777.0,75.770868,0.83215,31.010000,25.2,4.993941e+10,NaN
1473,ZAF,2018,NaN,1.670823e+10,NaN,5.16,23.129999,71.709999,1.112854e+11,2.405,...,NaN,NaN,NaN,31810.0,76.683188,NaN,31.559999,24.1,5.572291e+10,9.52
1474,ZAF,2019,NaN,1.554886e+10,NaN,5.28,22.309999,72.410004,1.060698e+11,2.381,...,95.022972,NaN,NaN,30937.0,77.584480,NaN,32.459999,23.5,5.522342e+10,NaN
1475,ZAF,2020,NaN,8.404204e+09,NaN,NaN,NaN,NaN,9.317915e+10,2.358,...,NaN,NaN,NaN,30153.0,78.474611,NaN,32.400002,NaN,NaN,NaN


In [24]:
list(df5)

['Country Code',
 'Date',
 'CPIA gender equality rating (1=low to 6=high)',
 'Commercial service exports (current US$)',
 'Electricity production from renewable sources, excluding hydroelectric (kWh)',
 'Employment in agriculture (% of total employment) (modeled ILO estimate)',
 'Employment in industry (% of total employment) (modeled ILO estimate)',
 'Employment in services (% of total employment) (modeled ILO estimate)',
 'Exports of goods and services (current US$)',
 'Fertility rate, total (births per woman)',
 'Foreign direct investment, net (BoP, current US$)',
 'GDP (current US$)',
 'Government expenditure on education, total (% of government expenditure)',
 'Labor force with advanced education (% of total working-age population with advanced education)',
 'Literacy rate, adult total (% of people ages 15 and above)',
 'Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)',
 'Net migration',
 'Number of infant deaths',
 'Peop

As there are lots of indicators that have very similar meaning we have decided to select some indicators to perform the study (**Indicator group** = *Name of the selected indicator*):
- **GDP** = *GDP (current US$)*
- **Literacy** = *Literacy rate, adult total (% of people ages 15 and above)', 'Government expenditure on education, total (% of government expenditure)*
- **Migration** = *Net migration*
- **Exports** = *Commercial service exports (current US$)* & *Exports of goods and services (current US$)*
- **International trading** = *Taxes on international trade (current LCU)*
- **Fertility** = *Fertility rate, total (births per woman)*
- **Healthcare** = *People using at least basic sanitation services (% of population)*
- **Employment** = *Employment in agriculture (% of total employment) (modeled ILO estimate)*, *Employment in services (% of total employment) (modeled ILO estimate)* & *Employment in industry (% of total employment) (modeled ILO estimate)*
- **Renewable energy** = *Electricity production from renewable sources, excluding hydroelectric (kWh)*
- **Mortality** = *Number of infant deaths*
- **Outside investment** = *Foreign direct investment, net (BoP, current US$)*
- **Pollution** = *Mortality rate attributed to household and ambient air pollution, age-standardized (per 100,000 population)*
- **Alcoholism** = *Total alcohol consumption per capita (liters of pure alcohol, projected estimates, 15+ years of age)*
- **Tech adoption** = *Research and development expenditure (% of GDP)*
- ** ** = *Labor force with advanced education (% of total working-age population with advanced education)*
- **Optimisim and pessimisim** = *Suicide mortality rate (per 100,000 population)*
- **Gender inequality** = *CPIA gender equality rating (**1=low to **6=high)*
- **Education** = *Share of youth not in education, employment or training, total (% of youth population)* & *Government expenditure on education, total (% of government expenditure)'*

To acomplish this, we use the function `isin` that will allow us to only select the the indicators afromentioned, that have been compilied in the list called *indicators_list*

In [25]:
df5.columns=['Country','Year','Gender equality','Exports-Commercial services','Renewable electricity','Employment-agriculture','Employment-industry','Employment-services','Exports-G&S','Fertility rate','Foreign investment','GDP','Education GExp','Workers high education','Literacy rate','Mortality-pollution','Net migration','Mortality-infants','Health services use','R&D GExp','Ninis','Suicide','International taxes','Alcohol per capita']

In [26]:
list(df5)
df5

,Country,Year,Gender equality,Exports-Commercial services,Renewable electricity,Employment-agriculture,Employment-industry,Employment-services,Exports-G&S,Fertility rate,...,Literacy rate,Mortality-pollution,Net migration,Mortality-infants,Health services use,R&D GExp,Ninis,Suicide,International taxes,Alcohol per capita
0,ARE,1990,NaN,NaN,0.0,NaN,NaN,NaN,NaN,4.454,...,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ARE,1991,NaN,NaN,0.0,8.46,33.330002,58.200001,NaN,4.253,...,NaN,NaN,NaN,645.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ARE,1992,NaN,NaN,0.0,8.37,33.360001,58.279999,NaN,4.041,...,NaN,NaN,368126.0,618.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ARE,1993,NaN,NaN,0.0,8.24,33.470001,58.290001,NaN,3.827,...,NaN,NaN,NaN,592.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,1994,NaN,NaN,0.0,8.13,33.490002,58.380001,NaN,3.618,...,NaN,NaN,NaN,568.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,ZAF,2017,NaN,1.614806e+10,NaN,5.28,23.340000,71.379997,1.042884e+11,2.430,...,87.046669,NaN,727026.0,32777.0,75.770868,0.83215,31.010000,25.2,4.993941e+10,NaN
1473,ZAF,2018,NaN,1.670823e+10,NaN,5.16,23.129999,71.709999,1.112854e+11,2.405,...,NaN,NaN,NaN,31810.0,76.683188,NaN,31.559999,24.1,5.572291e+10,9.52
1474,ZAF,2019,NaN,1.554886e+10,NaN,5.28,22.309999,72.410004,1.060698e+11,2.381,...,95.022972,NaN,NaN,30937.0,77.584480,NaN,32.459999,23.5,5.522342e+10,NaN
1475,ZAF,2020,NaN,8.404204e+09,NaN,NaN,NaN,NaN,9.317915e+10,2.358,...,NaN,NaN,NaN,30153.0,78.474611,NaN,32.400002,NaN,NaN,NaN


Get the mean of each column by country.

In [28]:
mean_value=df5.groupby('Country').mean()
mean_value

,Year,Gender equality,Exports-Commercial services,Renewable electricity,Employment-agriculture,Employment-industry,Employment-services,Exports-G&S,Fertility rate,Foreign investment,...,Literacy rate,Mortality-pollution,Net migration,Mortality-infants,Health services use,R&D GExp,Ninis,Suicide,International taxes,Alcohol per capita
Country,,,,,,,,,,,,,,,,,,,,,
ARE,2005.0,NaN,NaN,2.680769e+07,5.204483,34.332759,60.463448,2.529880e+11,2.420065,NaN,...,93.795387,54.7,1.019762e+06,597.096774,98.121723,0.863376,NaN,7.570,5.687960e+07,3.4120
ARG,2005.0,NaN,8.086058e+09,1.175615e+09,0.720345,24.454483,74.825172,4.985330e+10,2.501677,-5.914572e+09,...,98.618455,26.6,-7.066667e+04,11383.838710,91.407733,0.493970,19.618571,8.890,5.575804e+10,9.1400
AUT,2005.5,NaN,6.063244e+10,3.884692e+09,5.635517,28.200690,66.165518,1.505921e+11,1.433226,6.588530e+09,...,NaN,15.3,2.028288e+05,370.741935,99.985922,2.473488,7.603529,16.685,1.175665e+08,12.4280
AZE,2005.5,4.0000,2.007968e+09,1.019231e+07,39.469655,12.764138,47.767587,1.427552e+10,2.062371,-4.075839e+08,...,99.684505,63.9,-2.039367e+04,7774.548387,82.292682,0.254667,NaN,4.190,4.716260e+08,3.4440
BGD,2005.5,3.5625,1.162060e+09,1.719231e+07,54.253103,14.862414,30.887586,1.640451e+10,2.877548,-8.497250e+08,...,61.730904,149.0,-1.739225e+06,183494.903226,38.960588,NaN,28.298333,4.390,2.017016e+11,0.1180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,2005.5,NaN,4.412045e+11,1.242732e+11,1.544138,22.442414,76.014828,1.478503e+12,1.955984,7.536710e+09,...,NaN,13.3,5.637184e+06,27441.483871,99.821926,2.648437,13.172857,13.220,2.930337e+10,9.5460
VEN,2005.0,NaN,1.402185e+09,0.000000e+00,9.685172,21.785172,68.531035,4.751239e+10,2.698032,-1.187593e+09,...,94.567748,34.6,-6.545088e+05,10829.709677,94.333506,0.243271,16.512000,3.520,NaN,7.6400
VNM,2005.5,4.5000,7.081860e+09,4.561538e+07,55.376896,17.747931,26.876897,8.023430e+10,2.239065,-6.370960e+09,...,92.427185,64.5,-4.768672e+05,36290.838710,71.100662,0.344926,10.304615,6.940,NaN,5.2520


Fill null values of each indicator of each country with the mean computed previously.

In [30]:
df5.fillna(value=mean_value)

,Country,Year,Gender equality,Exports-Commercial services,Renewable electricity,Employment-agriculture,Employment-industry,Employment-services,Exports-G&S,Fertility rate,...,Literacy rate,Mortality-pollution,Net migration,Mortality-infants,Health services use,R&D GExp,Ninis,Suicide,International taxes,Alcohol per capita
0,ARE,1990,NaN,NaN,0.0,NaN,NaN,NaN,NaN,4.454,...,NaN,NaN,NaN,672.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ARE,1991,NaN,NaN,0.0,8.46,33.330002,58.200001,NaN,4.253,...,NaN,NaN,NaN,645.0,NaN,NaN,NaN,NaN,NaN,NaN
2,ARE,1992,NaN,NaN,0.0,8.37,33.360001,58.279999,NaN,4.041,...,NaN,NaN,368126.0,618.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ARE,1993,NaN,NaN,0.0,8.24,33.470001,58.290001,NaN,3.827,...,NaN,NaN,NaN,592.0,NaN,NaN,NaN,NaN,NaN,NaN
4,ARE,1994,NaN,NaN,0.0,8.13,33.490002,58.380001,NaN,3.618,...,NaN,NaN,NaN,568.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1472,ZAF,2017,NaN,1.614806e+10,NaN,5.28,23.340000,71.379997,1.042884e+11,2.430,...,87.046669,NaN,727026.0,32777.0,75.770868,0.83215,31.010000,25.2,4.993941e+10,NaN
1473,ZAF,2018,NaN,1.670823e+10,NaN,5.16,23.129999,71.709999,1.112854e+11,2.405,...,NaN,NaN,NaN,31810.0,76.683188,NaN,31.559999,24.1,5.572291e+10,9.52
1474,ZAF,2019,NaN,1.554886e+10,NaN,5.28,22.309999,72.410004,1.060698e+11,2.381,...,95.022972,NaN,NaN,30937.0,77.584480,NaN,32.459999,23.5,5.522342e+10,NaN
1475,ZAF,2020,NaN,8.404204e+09,NaN,NaN,NaN,NaN,9.317915e+10,2.358,...,NaN,NaN,NaN,30153.0,78.474611,NaN,32.400002,NaN,NaN,NaN


Group data by country.

In [31]:
groups=dict(tuple(df5.groupby('Country')))

Get a description of each indicator in each country.

In [34]:
df5.groupby('Country').describe(exclude=['Year'])

TypeError: data type 'Year' not understood

In [39]:
Q1=df5.groupby('Country').quantile(0.25)
Q3=df5.groupby('Country').quantile(0.75)
IQR=Q3-Q1
IQR

,Year,Gender equality,Exports-Commercial services,Renewable electricity,Employment-agriculture,Employment-industry,Employment-services,Exports-G&S,Fertility rate,Foreign investment,...,Literacy rate,Mortality-pollution,Net migration,Mortality-infants,Health services use,R&D GExp,Ninis,Suicide,International taxes,Alcohol per capita
Country,,,,,,,,,,,,,,,,,,,,,
ARE,15.0,NaN,NaN,0.000000e+00,5.130000,1.449997,3.830002,2.425265e+11,1.3050,NaN,...,3.761536,0.0,722279.00,116.0,1.232020,0.270060,NaN,1.450,1.232400e+08,0.640
ARG,15.0,NaN,8.825501e+09,1.330500e+09,0.600000,2.119999,2.210007,4.619173e+10,0.3160,6.391171e+09,...,0.242805,0.0,115500.00,5190.5,4.213969,0.143340,0.710001,0.650,7.251812e+10,0.800
AUT,15.5,NaN,1.268950e+10,4.716750e+09,1.650000,4.000000,5.589996,1.394271e+11,0.0750,8.692295e+09,...,NaN,0.0,97487.75,150.0,0.017996,0.930893,0.530000,1.325,4.297500e+06,0.720
AZE,15.5,0.000,3.688174e+09,0.000000e+00,3.669998,2.410000,2.430000,2.234833e+10,0.1650,9.221745e+08,...,0.021219,0.0,111903.25,6119.0,23.121518,0.105225,NaN,0.525,5.826268e+08,1.440
BGD,15.5,0.625,1.203719e+09,0.000000e+00,19.100002,7.349999,11.380001,2.261963e+10,1.2165,1.316865e+09,...,15.032219,0.0,1208525.25,145734.0,15.517397,NaN,2.995000,1.350,1.598132e+11,0.139
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,15.5,NaN,4.356840e+11,7.620650e+10,0.340000,5.010000,5.370003,1.217164e+12,0.1780,1.245140e+11,...,NaN,0.0,584339.50,5290.0,0.116263,0.194780,1.597500,2.200,1.466550e+10,0.460
VEN,15.0,NaN,4.170000e+08,0.000000e+00,2.910000,2.480000,5.209999,5.140853e+10,0.5375,1.921500e+09,...,2.183582,0.0,364245.00,3049.0,1.362653,0.108512,6.549999,2.150,NaN,2.330
VNM,15.5,0.000,7.763000e+09,7.725000e+07,17.880001,9.200001,9.330000,1.220568e+11,0.3530,7.608000e+09,...,3.831047,0.0,192572.25,12748.5,19.117037,0.248400,0.830001,1.225,NaN,5.230


In [40]:
df_out=df5.groupby('Country')[~((df5<(Q1-1.5*IQR))|(df5.groupby('Country')>(Q3+1.5*IQR))).any(axis=1)]

ValueError: Can only compare identically-labeled DataFrame objects